# Model Evaluation

Comparing James' (HealthServe Bot) results based on different parameters tested. <br>

MMR / Similarity Threshold = 0.5 / 0.8 <br>
K (number of source documents) = 5 / 10

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read the UAT data into a DataFrame named uat_df
uat_df = pd.read_excel('UAT.xlsx')
uat_df.sample(5)


,Question,Bloom's Taxonomy,Question.1,Expected answer,source doc,source chunck,MMR,Similarity threshold,k value,Jame's ans,cosine-similarity matrix,Best way?,Best k
10,3,Apply,What are some cases in which WICA can be clai...,Employee met with a traffic accident while tak...,Work Injury Compensation Act (by MOM) - Comple...,1-5. WICA 2019 AT A GLANCE Who is covered? All...,NaN,0.5,5,Under the Work Injury Compensation Act (WICA) ...,NaN,NaN,NaN
2,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-5. What is the Primary Care Plan (PCP) You m...,NaN,0.8,5,The Personal Care Assistant (PCA) program is d...,NaN,NaN,NaN
18,4,Analyze,What is the difference between WICA and PCP,PCP is a plan that provides eligible workers w...,Primary Care Plan (by MOM).pdf,1-10. What is the Primary Care Plan (PCP) You ...,NaN,0.8,10,WICA stands for the Work Injury Compensation A...,NaN,NaN,NaN
12,3,Apply,What are some cases in which WICA can be clai...,Employee met with a traffic accident while tak...,Work Injury Compensation Act (by MOM) - Comple...,1-5. WICA 2019 AT A GLANCE Who is covered? All...,NaN,0.8,5,Under the Work Injury Compensation Act (WICA) ...,NaN,NaN,NaN
0,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-3 . What is the Primary Care Plan (PCP) You ...,0.0,0.5,5,The Primary Care Plan (PCP) is mandatory for W...,NaN,mmr (as it can take in and understand any lang...,5.0


In [3]:
uat_df.isnull().sum()

Question                     0
Bloom's Taxonomy             0
Question.1                   0
Expected answer              0
source doc                   0
source chunck                0
MMR                         19
Similarity threshold         5
k value                      0
Jame's ans                   0
cosine-similarity matrix    25
Best way?                   24
Best k                      24
dtype: int64

In [4]:
# change NaN to 0 in mmr column
uat_df['MMR'] = uat_df['MMR'].fillna(0)

# drop last 3 columns
uat_df = uat_df.drop(uat_df.columns[[10, 11, 12]], axis=1)

uat_df.head()

,Question,Bloom's Taxonomy,Question.1,Expected answer,source doc,source chunck,MMR,Similarity threshold,k value,Jame's ans
0,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-3 . What is the Primary Care Plan (PCP) You ...,0.0,0.5,5,The Primary Care Plan (PCP) is mandatory for W...
1,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-7. What is the Primary Care Plan (PCP) You m...,0.0,0.5,10,The Primary Care Plan (PCP) is mandatory for W...
2,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-5. What is the Primary Care Plan (PCP) You m...,0.0,0.8,5,The Personal Care Assistant (PCA) program is d...
3,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-10. What is the Primary Care Plan (PCP) You ...,0.0,0.8,10,The Personal Care Assistant (PCA) program is d...
4,1,Remember,Who is required to be on the PCP?,PCP is mandatory for Work Permit (excluding mi...,Primary Care Plan (by MOM).pdf,1-5. What is the Primary Care Plan (PCP) You m...,1.0,NaN,5,The Primary Care Plan (PCP) is mandatory for W...


In [5]:
# for every data row, calculate cosine similarity between expected answer & james answer

# Import libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define a function that takes in two strings and calculates their cosine similarity
def calculate_cosine_similarity(string1, string2):
    # Create a CountVectorizer object
    count_vectorizer = CountVectorizer()
    # Transform the input strings using count_vectorizer
    count_vectorizer.fit([string1, string2])
    string1_vector = count_vectorizer.transform([string1])
    string2_vector = count_vectorizer.transform([string2])
    # Calculate the cosine similarity between the two vectors
    cosine = cosine_similarity(string1_vector, string2_vector)[0][0]
    return cosine


In [6]:
# Define a function that takes in a row and calculates the cosine similarity between the expected answer and James's answer
# add the cosine value to the row under a column named Cosine Similarity
for index, row in uat_df.iterrows():
    uat_df.loc[index, 'Cosine Similarity'] = calculate_cosine_similarity(row['Expected answer'], row["Jame's ans"])

In [7]:
uat_df.sample(4)

,Question,Bloom's Taxonomy,Question.1,Expected answer,source doc,source chunck,MMR,Similarity threshold,k value,Jame's ans,Cosine Similarity
8,2,Understand,How to explain PCP in a simple way for migrant...,The PCP aims to provide eligible workers with ...,Primary Care Plan (by MOM).pdf,1-10. What is the Primary Care Plan (PCP) You ...,0.0,0.8,10,PCP stands for Primary Care Partnership. It is...,0.482362
21,5,Evaluate,A migrant worker has sustained an injury in th...,He should claim his insurance through work inj...,2020 SJM Reach Paper “Are migrant workers in S...,"1-10. Act (WICA), which mandates employers to ...",0.0,0.5,10,The migrant worker should claim his insurance ...,0.603250
11,3,Apply,What are some cases in which WICA can be clai...,Employee met with a traffic accident while tak...,Work Injury Compensation Act (by MOM) - Comple...,1-9. WICA 2019 AT A GLANCE Who is covered? All...,0.0,0.5,10,Under the Work Injury Compensation Act 2019 (W...,0.310631
12,3,Apply,What are some cases in which WICA can be clai...,Employee met with a traffic accident while tak...,Work Injury Compensation Act (by MOM) - Comple...,1-5. WICA 2019 AT A GLANCE Who is covered? All...,0.0,0.8,5,Under the Work Injury Compensation Act (WICA) ...,0.280202


In [7]:
# Read output data into a excel
uat_df.to_excel('output.xlsx', index=False)

In [8]:
# Calculate average mean of cosine similarity between MMR and Similarity Search
uat_df.groupby('MMR')['Cosine Similarity'].mean()

MMR
0.0    0.518847
1.0    0.564454
Name: Cosine Similarity, dtype: float64

In [9]:
# From MMR, group by k and calculate average mean of cosine similarity
uat_df.groupby(['MMR', 'k value'])['Cosine Similarity'].mean()

MMR  k value
0.0  5          0.517977
     10         0.519716
1.0  5          0.564454
Name: Cosine Similarity, dtype: float64

In [11]:
# get average cosine similarity for similarity threshold 0.5 VS 0.8 across all bloom's taxonomy
uat_df.groupby(['Similarity threshold', "Bloom's Taxonomy"])['Cosine Similarity'].mean()

Similarity threshold  Bloom's Taxonomy
0.5                   Analyze             0.587662
                      Apply               0.286207
                      Evaluate            0.603250
                      Remember            0.872110
                      Understand          0.569361
0.8                   Analyze             0.582183
                      Apply               0.232701
                      Evaluate            0.603250
                      Remember            0.382506
                      Understand          0.469237
Name: Cosine Similarity, dtype: float64

In [12]:
# average cosine similarity for mmr VS similarity across all bloom's taxonomy
uat_df.groupby(['MMR', "Bloom's Taxonomy"])['Cosine Similarity'].mean()

MMR  Bloom's Taxonomy
0.0  Analyze             0.584923
     Apply               0.259454
     Evaluate            0.603250
     Remember            0.627308
     Understand          0.519299
1.0  Analyze             0.564576
     Apply               0.280202
     Evaluate            0.603250
     Remember            0.872110
     Understand          0.502130
Name: Cosine Similarity, dtype: float64

### Evaluation Part 2

We will use cosine similarity matrix to evaluate who is best: James/ ChatGPT/ Perplexity

In [11]:
# Read the UAT2 data into a DataFrame named uat_df
uat2_df = pd.read_excel('UAT2.xlsx')
uat2_df.sample(5)

,Index,Bloom's Taxonomy Category,Question,Expected Answer,Parameters,James,ChatGPT,Perplexity,Which is better (qualitative)?,Which is better (quantitatively)?,Comment
33,34,Analyze,compare the ailments that are eligible or inel...,Medical expenses related to work injury for mi...,"mmr, k=5","Under the Work Injury Compensation Act (WICA),...",The Work Injury Compensation Act (WICA) in Sin...,The Work Injury Compensation Act (WICA) covers...,ChatGPT,NaN,NaN
5,6,Understand,Can you clarify if PCP cover dental services?,PCP does not cover dental services. \nWorkers ...,"mmr, k=5","No, the Primary Care Plan (PCP) does not cover...","The term ""PCP"" can have various meanings depen...","Based on the search results, it is unclear whe...",James,NaN,NaN
13,14,Evaluate,What is the level of awareness among migrant w...,In a research survey conducted with 433 non-do...,"mmr, k=5",According to a study conducted with non-domest...,Language Barriers: Many migrant workers in Sin...,"According to the search results, there are sev...",Perplexity,NaN,NaN
34,35,Analyze,compare the requirements for insurance coverag...,"Under EFMA, MOM (n.d.) mandates that employers...","mmr, k=5",Both EFMA and WICA provide insurance coverage ...,EFMA (Enhanced Fund for the Mind) and WICA (Wo...,The Employment of Foreign Manpower Act (EFMA) ...,ChatGPT,NaN,NaN
32,33,Analyze,If I want to get help for a migrant workers' m...,HealthServe provides the most coverage\nHealth...,"mmr, k=5",If you want to get help for a migrant worker's...,When it comes to seeking help for a migrant wo...,Mental health is an important aspect of overal...,Perplexity,NaN,NaN


In [12]:
# Drop last 3 columns
uat2_df = uat2_df.drop(uat2_df.columns[[8,9,10]], axis=1)

In [13]:
uat2_df.isnull().sum()

Index                        0
Bloom's Taxonomy Category    0
Question                     0
Expected Answer              0
Parameters                   0
James                        0
ChatGPT                      0
Perplexity                   0
dtype: int64

In [14]:
# Compare cosine similarity between James and Expected Answer, repeat for Perplexity & ChatGPT
for index, row in uat2_df.iterrows():
    uat2_df.loc[index, 'James Similarity'] = calculate_cosine_similarity(row['Expected Answer'], row["James"])
    uat2_df.loc[index, 'Perplexity Similarity'] = calculate_cosine_similarity(row['Expected Answer'], row['Perplexity'])
    uat2_df.loc[index, 'ChatGPT Similarity'] = calculate_cosine_similarity(row['Expected Answer'], row['ChatGPT'])


# Bot with highest cosine similarity is the best quantitatively, add column for that and iterrate through every row
for index, row in uat2_df.iterrows():
    if row['James Similarity'] > row['Perplexity Similarity'] and row['James Similarity'] > row['ChatGPT Similarity']:
        uat2_df.loc[index, 'Best Bot (quantitatively)'] = 'James'
    elif row['Perplexity Similarity'] > row['James Similarity'] and row['Perplexity Similarity'] > row['ChatGPT Similarity']:
        uat2_df.loc[index, 'Best Bot (quantitatively)'] = 'Perplexity'
    elif row['ChatGPT Similarity'] > row['James Similarity'] and row['ChatGPT Similarity'] > row['Perplexity Similarity']:
        uat2_df.loc[index, 'Best Bot (quantitatively)'] = 'ChatGPT'
    else:
        uat2_df.loc[index, 'Best Bot (quantitatively)'] = 'Tie'

# Read output data into a excel
uat2_df.to_excel('output2.xlsx', index=False)

In [19]:
# Calculate which "Best Bot (quantitatively)" has the highest count
uat2_df.groupby('Best Bot (quantitatively)').count()

,Index,Bloom's Taxonomy Category,Question,Expected Answer,Parameters,James,ChatGPT,Perplexity,James Similarity,Perplexity Similarity,ChatGPT Similarity
Best Bot (quantitatively),,,,,,,,,,,
ChatGPT,13,13,13,13,13,13,13,13,13,13,13
James,13,13,13,13,13,13,13,13,13,13,13
Perplexity,10,10,10,10,10,10,10,10,10,10,10


In [20]:
# Calculate average mean of cosine similarity by blooms taxonomy category
uat2_df.groupby("Bloom's Taxonomy Category")['James Similarity', 'Perplexity Similarity', 'ChatGPT Similarity'].mean()

/var/folders/j2/fdyj63p12l7gh5_swn1pdqgc0000gn/T/ipykernel_99879/613326895.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  uat2_df.groupby("Bloom's Taxonomy Category")['James Similarity', 'Perplexity Similarity', 'ChatGPT Similarity'].mean()


,James Similarity,Perplexity Similarity,ChatGPT Similarity
Bloom's Taxonomy Category,,,
Analyze,0.516545,0.585054,0.571982
Apply,0.537877,0.459945,0.423247
Create,0.452329,0.474875,0.509364
Evaluate,0.498401,0.492870,0.513572
Remember,0.574755,0.496220,0.579041
Understand,0.523692,0.421703,0.405849
